## Default Setting

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Okt
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
#git hub test

## URL Parsing

In [2]:
def get_naver_economy_news_urls_from_list(pages=5, max_urls=50):
    base_url = "https://news.naver.com/main/list.naver"
    headers = {"User-Agent": "Mozilla/5.0"}
    all_links = set()

    for page in range(1, pages + 1):
        params = {
            "mode": "LSD",
            "mid": "shm",
            "sid1": "101",  # 경제 뉴스 섹션
            "page": str(page)
        }
        response = requests.get(base_url, headers=headers, params=params)
        print(f"{page}페이지 응답 코드: {response.status_code}")

        if response.status_code != 200:
            print(f"{page}페이지 요청 실패")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        for a in soup.select("dt > a"):
            href = a.get("href")
            if href and href.startswith("https://n.news.naver.com"):
                all_links.add(href)
                if len(all_links) >= max_urls:
                    print(f"최대 {max_urls}개 링크 수집 완료, 수집 종료")
                    return list(all_links)

    print(f"총 {len(all_links)}개의 중복 없는 링크를 수집했습니다.")
    return list(all_links)

## Word crawling

In [3]:
def get_news_title(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("❌ 뉴스 기사 크롤링 실패:", response.status_code)
        return ""

    soup = BeautifulSoup(response.text, "html.parser")
    
    title_tag = soup.select_one("h2#title") or soup.select_one("h3#articleTitle")
    if title_tag:
        return title_tag.get_text(strip=True)

    # <title> 태그 사용 (네이버 뉴스는 보통 '기사제목 - 네이버 뉴스' 형태)
    if soup.title:
        full_title = soup.title.get_text()
        return full_title.replace(" - 네이버 뉴스", "").strip()
    
    print("❌ 뉴스 제목을 찾을 수 없음")
    return ""


## CSV파일 저장

In [4]:
def save_news_to_csv(filename, texts):
    with open(filename, mode='w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        # 헤더 작성
        writer.writerow(["text", "category"])  
        # category는 빈 칸으로 둠 (나중에 수동 입력)
        for text in texts:
            writer.writerow([text, ""])  
            
            


## 크롤링 & 단어 matching test

In [5]:
if __name__ == "__main__":
    news_urls = get_naver_economy_news_urls_from_list(1, max_urls=1)
    titles = []
    for url in news_urls:
        title = get_news_title(url)
        if title:
            titles.append(title)
    
    save_news_to_csv("news_data.csv", titles)


1페이지 응답 코드: 200
최대 1개 링크 수집 완료, 수집 종료


PermissionError: [Errno 13] Permission denied: 'news_data.csv'

## 데이터 학습

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. 데이터 불러오기 및 전처리
df = pd.read_csv("news_data.csv", encoding="euc-kr")
df["category"] = df["category"].str.strip()  # 공백 제거
df = df[df["category"].notnull()]  # NaN 제거

X = df["text"]
y = df["category"]

# 2. TF-IDF 벡터화 (성능 개선을 위해 ngram 추가)
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_vec = vectorizer.fit_transform(X)

# 3. 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

# 4. 로지스틱 회귀 모델 학습 (클래스 가중치 적용)
model = LogisticRegression(max_iter=1000, class_weight="balanced")
model.fit(X_train, y_train)

# 5. 성능 평가
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))  # zero_division=0으로 경고 제거

# 6. 새 기사 분류 함수
def classify_article(text):
    X_vec = vectorizer.transform([text])
    return model.predict(X_vec)[0]

# 예시 테스트
print("분류 결과:", classify_article("정부, 주택담보대출 규제 완화 검토…수요 회복 기대"))


              precision    recall  f1-score   support

          국제       1.00      0.25      0.40         4
          금융       0.83      0.38      0.53        13
          기타       1.00      0.25      0.40         4
         부동산       0.87      0.59      0.70        22
          사회       0.00      0.00      0.00         3
          산업       0.31      1.00      0.47        14
          소비       1.00      0.50      0.67         4
          정책       1.00      0.62      0.77        16

    accuracy                           0.57        80
   macro avg       0.75      0.45      0.49        80
weighted avg       0.78      0.57      0.59        80

분류 결과: 부동산


In [8]:
# 예시 기사 10개
test_articles = [
    "삼성전자, 차세대 반도체 공정 개발 완료…양산 준비 돌입",
    "정부, 주택담보대출 규제 완화 검토…수요 회복 기대",
    "서울 아파트 전셋값 3개월 연속 상승…전세난 재현되나",
    "한은, 기준금리 동결…물가 안정 기조 유지",
    "현대차, 미국에 전기차 공장 신설…총 10조 원 투자",
    "중소기업청, 청년 창업 지원금 200억 원 추가 투입",
    "카카오뱅크, 2분기 순이익 1200억…사상 최대 실적",
    "롯데건설, 부산 해운대 대형 주거단지 수주",
    "미국 금리 인상 영향…원/달러 환율 1,350원 돌파",
    "정부, 소비 회복 위해 추석 전 재난지원금 지급 추진"
]

# 분류 함수 (이전과 동일)
def classify_article(text):
    X_vec = vectorizer.transform([text])
    return model.predict(X_vec)[0]

# 결과 출력
for i, article in enumerate(test_articles, 1):
    category = classify_article(article)
    print(f"{i}. \"{article}\" → 분류 결과: {category}")


1. "삼성전자, 차세대 반도체 공정 개발 완료…양산 준비 돌입" → 분류 결과: 산업
2. "정부, 주택담보대출 규제 완화 검토…수요 회복 기대" → 분류 결과: 정책
3. "서울 아파트 전셋값 3개월 연속 상승…전세난 재현되나" → 분류 결과: 부동산
4. "한은, 기준금리 동결…물가 안정 기조 유지" → 분류 결과: 산업
5. "현대차, 미국에 전기차 공장 신설…총 10조 원 투자" → 분류 결과: 산업
6. "중소기업청, 청년 창업 지원금 200억 원 추가 투입" → 분류 결과: 산업
7. "카카오뱅크, 2분기 순이익 1200억…사상 최대 실적" → 분류 결과: 산업
8. "롯데건설, 부산 해운대 대형 주거단지 수주" → 분류 결과: 부동산
9. "미국 금리 인상 영향…원/달러 환율 1,350원 돌파" → 분류 결과: 산업
10. "정부, 소비 회복 위해 추석 전 재난지원금 지급 추진" → 분류 결과: 산업
